# Factorial design with Empirical Bayes and Thompson Sampling

<markdowncell>
This tutorial illustrates how to run a factorial experiment.
We use Empirical Bayes to model the data and Thompson Sampling to generate
arms.

In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
from typing import Dict, Optional, Tuple, Union
from ax import (
    ChoiceParameter,
    Arm,
    ParameterType,
    SearchSpace,
    SimpleExperiment,
    modelbridge,
)
from ax.metrics.factorial import (
    evaluation_function as factorial_metric_evaluation_function,
)
from ax.plot.scatter import plot_fitted
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.stats.statstools import agresti_coull_sem

In [2]:
init_notebook_plotting()

[INFO 04-01 09:59:07] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


<markdowncell>
First, we define our search space. In this example we have three parameters,
each of which has a set of possible string values.

In [3]:
search_space = SearchSpace(
    parameters=[
        ChoiceParameter(
            name="factor1",
            parameter_type=ParameterType.STRING,
            values=["level11", "level12", "level13"],
        ),
        ChoiceParameter(
            name="factor2",
            parameter_type=ParameterType.STRING,
            values=["level21", "level22"],
        ),
        ChoiceParameter(
            name="factor3",
            parameter_type=ParameterType.STRING,
            values=["level31", "level32", "level33", "level34"],
        ),
    ]
)

<markdowncell>
Second, we define an evaluation function, which is responsible for computing
the mean and standard error of a given parameterization. In this example,
each possible parameter value is given a weight, and the mean of a parameterization
is determined by the weights of its values. The higher the weights, the greater
the mean.

In [4]:
one_hot_encoder = skl.preprocessing.OneHotEncoder(
    categories=[par.values for par in search_space.parameters.values()], 
)

def factorial_evaluation_function(
    # `parameterization` is a dict of parameter names to values of those parameters.
    parameterization: Dict[str, Optional[Union[str, bool, float]]],
    # `weight` is the weight of the parameterization, which is used to determine the variance of the estimate
    weight: Optional[float] = None,
) -> Dict[str, Tuple[float, float]]:  # dict of metric names to tuple of mean and standard error.
    batch_size = 10000
    noise_level = 0.0
    weight = weight if weight is not None else 1.0
    coefficients = np.array([
        0.1, 0.2, 0.3,
        0.1, 0.2,
        0.1, 0.2, 0.3, 0.4
    ])
    features = np.array(list(parameterization.values())).reshape(1, -1)
    encoded_features = one_hot_encoder.fit_transform(features)
    z = coefficients @ encoded_features.T + np.sqrt(noise_level) * np.random.randn()
    p = np.exp(z) / (1 + np.exp(z))
    plays = np.random.binomial(batch_size, weight)
    successes = np.random.binomial(plays, p)
    mean = float(successes) / plays
    sem = agresti_coull_sem(successes, plays)
    return {
        "success_metric": (mean, sem)
    }

<markdowncell>
We now set up our experiment and define the status quo arm.

In [5]:
exp = SimpleExperiment(
    name="my_factorial_closed_loop_experiment",
    search_space=search_space,
    evaluation_function=factorial_evaluation_function,
    objective_name="success_metric",
)
exp.status_quo = Arm(
    params={"factor1": "level11", "factor2": "level21", "factor3": "level31"}
)

<markdowncell>
We generate an initial batch that explores the full space of the factorial
design, including the status quo. Note that the status quo gets a weight of 5x
relative to the other arms since it is within the search space of the
FullFactorial generator.

In [6]:
factorial_generator = modelbridge.get_factorial(search_space=exp.search_space)
factorial_run = factorial_generator.gen(n=-1)
trial = (
    exp.new_batch_trial()
    .add_generator_run(factorial_run, multiplier=1)
    .add_arms_and_weights(arms=[exp.status_quo], multiplier=4)
)

<markdowncell>
We use Thompson sampling with batched updates to give more weight to best-performing arms while balancing exploration.

In [7]:
generators = []
for i in range(4):
    print("Running iteration {}...".format(i))
    data = exp.eval_trial(trial)
    thompson_generator = modelbridge.get_thompson(
        experiment=exp, data=data, min_weight=0.01
    )
    generators.append(thompson_generator)
    thompson_run = thompson_generator.gen(n=-1)
    trial = exp.new_batch_trial().add_generator_run(thompson_run)

Running iteration 0...
Running iteration 1...
Running iteration 2...
Running iteration 3...


<markdowncell>
The following plots shows the predicted value of our objective for the arms in each trial. Over time, we hone in on the parameterizations with the highest values. 

In [8]:
for generator in generators:
    render(plot_fitted(generator, metric="success_metric", rel=False))

<markdowncell>
As expected given our evaluation function, parameterizations with higher levels
perform better and are given higher weight. Below we see the parameterizations 
that made it to the final trial (along with the status quo).

In [9]:
results = pd.DataFrame(
    [
        {"values": ",".join(arm.params.values()), "weight": weight}
        for arm, weight in trial.normalized_arm_weights().items()
    ]
)
print(results)

                    values    weight
0  level13,level22,level34  0.447076
1  level13,level21,level34  0.219591
2  level11,level21,level31  0.333333


In [10]:
from ax.plot.bandit_rollout import plot_bandit_rollout
from ax.utils.notebook.plotting import render

render(plot_bandit_rollout(exp))

In [12]:
from ax.plot.marginal_effects import plot_marginal_effects
render(plot_marginal_effects(generators[0], 'success_metric'))